In [ ]:
#주요 모듈 임포트
import requests
from bs4 import BeautifulSoup
from urllib import request

In [ ]:
page = 1

comp_list=[]
date_list=[]

#총 페이지 설정
total_page = 116

while True:
    url = 'https://finance.naver.com/research/debenture_list.nhn?' #주소
    
    #파라메터 선언
    params = {
        'page' : page
    }

    #수프질
    resp = requests.get(url, params=params)
    soup = BeautifulSoup(resp.content, 'html.parser')

    #링크덜
    td_link = soup.find('table', class_='type_1').find_all('td', class_='file')
    td_date = soup.find('table', class_='type_1').find_all('td', class_='date')
    tr_comp = soup.find('table', class_= 'type_1').find_all('tr')

    #증권사 명 추출
    
    for tr in tr_comp:
        try:
            comp=tr.find_all('td')[1].text
            comp_list.append(comp)
            print(f'{comp}', end = ' ')
        except:
            continue
    
    #날짜 추출
    
    for idx, td in enumerate(td_date):
        if idx % 2 == 0:
            date_list.append(td.text)
            print(f'{td.text} 읽음', end=' ')

    #파일 다운로드, 파일 존재 시 돌리지 말 것.
#     for idx, td in enumerate(td_link):
#         file_link = td.find('a')['href']
#         fname=f'{page}_{idx+1}.pdf'
#         request.urlretrieve(file_link, fname)
#         print(f'{comp_list[idx]} {date_list[idx]} {fname} 다운로드 완료')
        
    if page>=total_page : #페이지 다 넘기면 탈출
        break
        
    page+=1 #페이지 넘기기
    print()

In [ ]:
!pip install pdfminer.six

In [ ]:
def pdfToText(fname):

    from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
    from pdfminer.converter import TextConverter
    from pdfminer.layout import LAParams
    from pdfminer.pdfpage import PDFPage
    from io import StringIO
    
    try:
        rsrcmgr = PDFResourceManager()
        retstr = StringIO()
        codec = 'utf-8'
        laparams = LAParams()
        device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
        fp = open(f'rawReport/{fname}', 'rb')
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        password = ''
        maxpages = 0
        caching = True
        pagenos = set()

        for page in PDFPage.get_pages(fp, pagenos, maxpages = maxpages, password = password,
                                      caching = caching, check_extractable = True):
            interpreter.process_page(page)

        text = retstr.getvalue()

        fp.close()
        device.close()
        retstr.close()

        return text
    except Exception as e:
        return str(e)

In [ ]:
len(comp_list)

In [ ]:
len(date_list)

In [ ]:
import pandas as pd

text_list=[]
for idx, comp in enumerate(comp_list):
    
    page = (idx // 30)+1
    p_no = (idx % 30)+1
    
    text=pdfToText(f'{page}_{p_no}.pdf')
    
    print(f'{page}_{p_no}.pdf 텍스트 추출...')
    text=text.replace('\n','')
    text=text.replace('\r','')
    text=text.replace('\t','')
    
    text_list.append(text)
    
    if (idx+1) % 30 == 0:
        dict_report ={
            '증권사' : comp_list[idx-29:idx+1],
            '날짜' : date_list[idx-29:idx+1],
            '텍스트' : text_list
        }
        
        df = pd.DataFrame(dict_report)
        df.to_csv(f'bond_report page_{page}.csv', index=False)
        
        text_list = []


rest = len(text_list)

dict_report ={
            '증권사' : comp_list[len(comp_list)-rest:],
            '날짜' : date_list[len(date_list)-rest:],
            '텍스트' : text_list
            }
        
df = pd.DataFrame(dict_report)
df.to_csv(f'bond_report page_{page}.csv', index=False)